In [14]:
import pandas as pd
import numpy as np
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [17]:
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [94]:
df[df.Exited==0].shape

(7963, 13)

In [95]:
df[df.Exited==1].shape

(2037, 13)

In [33]:
df['Gender'].replace({'Female':0,'Male':1},inplace=True)

In [53]:
cols_to_scale=['CreditScore','Age','Balance','EstimatedSalary']

In [55]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
df[cols_to_scale]=scaler.fit_transform(df[cols_to_scale])

In [60]:
X=df.drop(['Exited'],axis=1)
y=df['Exited']

In [66]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.2)

In [67]:
X_train.shape

(8000, 12)

In [68]:
X_test.shape

(2000, 12)

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [97]:
def ANN(X_train,y_train,X_test,y_test,loss,weights):

    model=keras.Sequential([
        keras.layers.Dense(20,input_shape=(12,),activation='relu'),
        keras.layers.Dense(40,activation='relu'),
        keras.layers.Dense(80,activation='relu'),
        keras.layers.Dense(40,activation='relu'),
        keras.layers.Dense(20,activation='relu'),
        keras.layers.Dense(1,activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                  loss=loss,
                  metrics=['accuracy'])
    if weights==-1:
        model.fit(X_train,y_train,epochs=100)
    else:
        model.fit(X_train,y_train,epochs=100,class_weight=weights)
        
    print(model.evaluate(X_test,y_test))
    
    y_pred=model.predict(X_test)
    y_pred=np.round(y_pred)
    
    print('Classification Report: \n',classification_report(y_test,y_pred))
    
    return y_pred


In [98]:
y_pred=ANN(X_train,y_train,X_test,y_test,'binary_crossentropy',-1)

Epoch 1/100
8000/8000 [==============================] - 0s 49us/sample - loss: 0.4976 - acc: 0.7946
Epoch 2/100
8000/8000 [==============================] - 0s 33us/sample - loss: 0.4667 - acc: 0.7968
Epoch 3/100
8000/8000 [==============================] - 0s 33us/sample - loss: 0.4454 - acc: 0.8075
Epoch 4/100
8000/8000 [==============================] - 0s 33us/sample - loss: 0.4213 - acc: 0.8195
Epoch 5/100
8000/8000 [==============================] - 0s 33us/sample - loss: 0.4073 - acc: 0.8253
Epoch 6/100
8000/8000 [==============================] - 0s 34us/sample - loss: 0.3908 - acc: 0.8332
Epoch 7/100
8000/8000 [==============================] - 0s 33us/sample - loss: 0.3783 - acc: 0.8390
Epoch 8/100
8000/8000 [==============================] - 0s 35us/sample - loss: 0.3697 - acc: 0.8454
Epoch 9/100
8000/8000 [==============================] - 0s 38us/sample - loss: 0.3669 - acc: 0.8447
Epoch 10/100
8000/8000 [==============================] - 0s 37us/sample - loss: 0.3617 - a

In [82]:
yp=model.predict(X_test)

In [84]:
y_pred=[]
for element in yp:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
        

In [85]:
y_test[:5]

9394    0
898     1
2398    0
5906    0
2343    0
Name: Exited, dtype: int64

In [86]:
y_pred[:5]

[0, 0, 0, 0, 0]

In [88]:
from sklearn.metrics import confusion_matrix,classification_report
print (classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91      1595
           1       0.68      0.47      0.55       405

    accuracy                           0.85      2000
   macro avg       0.78      0.71      0.73      2000
weighted avg       0.84      0.85      0.84      2000



### Method 1 : Under Sampling

In [102]:
count_class_0,count_class_1=df.Exited.value_counts()


In [103]:
count_class_0

7963

In [104]:
count_class_1

2037

In [106]:
df_class_0=df[df['Exited']==0]
df_class_1=df[df['Exited']==1]

In [113]:
df_class_0_under=df_class_0.sample(count_class_1)

df_test_under=pd.concat([df_class_0_under,df_class_1],axis=0)
df_test_under.shape
print(df_test_under.Exited.value_counts())

1    2037
0    2037
Name: Exited, dtype: int64


In [118]:
X=df_test_under.drop('Exited',axis=1)
y=df_test_under['Exited']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=15,stratify=y,test_size=0.2)

In [119]:
X_train.shape

(3259, 12)

In [120]:
X_test.shape

(815, 12)

In [121]:
y_train.value_counts()

1    1630
0    1629
Name: Exited, dtype: int64

In [125]:
y_preds=ANN(X_train,y_train,X_test,y_test,'binary_crossentropy',-1)

Epoch 1/100
3259/3259 [==============================] - 0s 79us/sample - loss: 0.6831 - acc: 0.5492
Epoch 2/100
3259/3259 [==============================] - 0s 33us/sample - loss: 0.6552 - acc: 0.6272
Epoch 3/100
3259/3259 [==============================] - 0s 35us/sample - loss: 0.6167 - acc: 0.6643
Epoch 4/100
3259/3259 [==============================] - 0s 35us/sample - loss: 0.5867 - acc: 0.6956
Epoch 5/100
3259/3259 [==============================] - 0s 35us/sample - loss: 0.5641 - acc: 0.7076
Epoch 6/100
3259/3259 [==============================] - 0s 35us/sample - loss: 0.5309 - acc: 0.7324
Epoch 7/100
3259/3259 [==============================] - 0s 35us/sample - loss: 0.5106 - acc: 0.7469
Epoch 8/100
3259/3259 [==============================] - 0s 35us/sample - loss: 0.5033 - acc: 0.7518
Epoch 9/100
3259/3259 [==============================] - 0s 35us/sample - loss: 0.5023 - acc: 0.7542
Epoch 10/100
3259/3259 [==============================] - 0s 35us/sample - loss: 0.4955 - a

## Method 2 : Over Sampling

In [128]:
count_class_0,count_class_1

(7963, 2037)

In [132]:
df_class_1_over=df_class_1.sample(count_class_0,replace=True)

In [145]:
df_class_1_over.Exited.value_counts()

1    7963
Name: Exited, dtype: int64

In [144]:
df_class_0.Exited.value_counts()

0    7963
Name: Exited, dtype: int64

In [147]:
df_test_over=pd.concat([df_class_0,df_class_1_over],axis=0)
df_test_over.Exited.value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

In [148]:
X=df_test_over.drop('Exited',axis=1)
y=df_test_over['Exited']

In [155]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=15,stratify=y)

In [156]:
X_train.shape

(12740, 12)

In [157]:
y_test.shape

(3186,)

In [158]:
y_train.value_counts()

1    6370
0    6370
Name: Exited, dtype: int64

In [159]:
y_preds=ANN(X_train,y_train,X_test,y_test,'binary_crossentropy',-1)

Epoch 1/100
12740/12740 [==============================] - 1s 51us/sample - loss: 0.6408 - acc: 0.6351
Epoch 2/100
12740/12740 [==============================] - 0s 36us/sample - loss: 0.5466 - acc: 0.7243
Epoch 3/100
12740/12740 [==============================] - 0s 35us/sample - loss: 0.5070 - acc: 0.7542
Epoch 4/100
12740/12740 [==============================] - 0s 36us/sample - loss: 0.4963 - acc: 0.7581
Epoch 5/100
12740/12740 [==============================] - 0s 35us/sample - loss: 0.4855 - acc: 0.7629
Epoch 6/100
12740/12740 [==============================] - 0s 35us/sample - loss: 0.4772 - acc: 0.7675
Epoch 7/100
12740/12740 [==============================] - 0s 36us/sample - loss: 0.4745 - acc: 0.7673
Epoch 8/100
12740/12740 [==============================] - 0s 35us/sample - loss: 0.4681 - acc: 0.7728
Epoch 9/100
12740/12740 [==============================] - 0s 35us/sample - loss: 0.4603 - acc: 0.7755
Epoch 10/100
12740/12740 [==============================] - 0s 35us/sampl

### Method 3 : SMOTE

In [161]:
X=df.drop('Exited',axis='columns')
y=df['Exited']

In [165]:
y.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [168]:
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy='minority')
X_sm,y_sm=smote.fit_sample(X,y)

y_sm.value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

In [170]:
X_train,X_test,y_train,y_test=train_test_split(X_sm,y_sm,random_state=15,stratify=y_sm,test_size=0.2)

In [171]:
y_train.value_counts()

1    6370
0    6370
Name: Exited, dtype: int64

In [172]:
y_preds=ANN(X_train,y_train,X_test,y_test,'binary_crossentropy',-1)

Epoch 1/100
12740/12740 [==============================] - 1s 55us/sample - loss: 0.6234 - acc: 0.6460
Epoch 2/100
12740/12740 [==============================] - 0s 38us/sample - loss: 0.5266 - acc: 0.7370
Epoch 3/100
12740/12740 [==============================] - 0s 38us/sample - loss: 0.4863 - acc: 0.7646
Epoch 4/100
12740/12740 [==============================] - 0s 38us/sample - loss: 0.4697 - acc: 0.7748
Epoch 5/100
12740/12740 [==============================] - 0s 38us/sample - loss: 0.4600 - acc: 0.7808
Epoch 6/100
12740/12740 [==============================] - 0s 38us/sample - loss: 0.4494 - acc: 0.7862
Epoch 7/100
12740/12740 [==============================] - 0s 38us/sample - loss: 0.4356 - acc: 0.7951
Epoch 8/100
12740/12740 [==============================] - 0s 38us/sample - loss: 0.4337 - acc: 0.79850s - loss: 0.4344 - acc: 0.798
Epoch 9/100
12740/12740 [==============================] - 0s 39us/sample - loss: 0.4245 - acc: 0.7998
Epoch 10/100
12740/12740 [=================

Epoch 79/100
12740/12740 [==============================] - 1s 41us/sample - loss: 0.2445 - acc: 0.8931
Epoch 80/100
12740/12740 [==============================] - 1s 41us/sample - loss: 0.2426 - acc: 0.8966
Epoch 81/100
12740/12740 [==============================] - 1s 40us/sample - loss: 0.2340 - acc: 0.9018
Epoch 82/100
12740/12740 [==============================] - 1s 41us/sample - loss: 0.2418 - acc: 0.8961
Epoch 83/100
12740/12740 [==============================] - 1s 40us/sample - loss: 0.2369 - acc: 0.8985
Epoch 84/100
12740/12740 [==============================] - 1s 41us/sample - loss: 0.2411 - acc: 0.8955
Epoch 85/100
12740/12740 [==============================] - 1s 40us/sample - loss: 0.2346 - acc: 0.8994
Epoch 86/100
12740/12740 [==============================] - 1s 41us/sample - loss: 0.2325 - acc: 0.9005
Epoch 87/100
12740/12740 [==============================] - 1s 41us/sample - loss: 0.2312 - acc: 0.9000
Epoch 88/100
12740/12740 [==============================] - 1s 4

### Method 4: use of Ensemble with undersampling

In [221]:
df.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [222]:
df3_class0=df[df.Exited==0]
df3_class1=df[df.Exited==1]

In [223]:
df3_class0.shape

(7963, 13)

In [225]:
7963/2037

3.9091801669121256

In [226]:
df3_class1.shape

(2037, 13)

In [227]:
def get_train_batch(df_majority,df_minority,start,end):
    df_train=pd.concat([df_majority[start:end],df_minority],axis=0)
    
    X=df_train.drop('Exited',axis='columns')
    y=df_train['Exited']
    
    return X,y

In [228]:
X_en,y_en=get_train_batch(df3_class0,df3_class1,0,2037)
y_en.value_counts()

1    2037
0    2037
Name: Exited, dtype: int64

In [229]:
X_train,X_test,y_train,y_test=train_test_split(X_en,y_en,random_state=15,stratify=y_en,test_size=0.2)

In [230]:
y_pred1=ANN(X_train,y_train,X_test,y_test,'binary_crossentropy',-1)

Epoch 1/100
3259/3259 [==============================] - 0s 111us/sample - loss: 0.6840 - acc: 0.5483
Epoch 2/100
3259/3259 [==============================] - 0s 42us/sample - loss: 0.6401 - acc: 0.6358
Epoch 3/100
3259/3259 [==============================] - 0s 43us/sample - loss: 0.6077 - acc: 0.6729
Epoch 4/100
3259/3259 [==============================] - 0s 42us/sample - loss: 0.5869 - acc: 0.6812
Epoch 5/100
3259/3259 [==============================] - 0s 43us/sample - loss: 0.5638 - acc: 0.6968
Epoch 6/100
3259/3259 [==============================] - 0s 43us/sample - loss: 0.5494 - acc: 0.7088
Epoch 7/100
3259/3259 [==============================] - 0s 44us/sample - loss: 0.5287 - acc: 0.7358
Epoch 8/100
3259/3259 [==============================] - 0s 43us/sample - loss: 0.5150 - acc: 0.7487
Epoch 9/100
3259/3259 [==============================] - 0s 42us/sample - loss: 0.5153 - acc: 0.7423
Epoch 10/100
3259/3259 [==============================] - 0s 43us/sample - loss: 0.5013 - 

In [231]:
X_en_1,y_en_1=get_train_batch(df3_class0,df3_class1,2038,4074)
y_en_1.value_counts()

1    2037
0    2036
Name: Exited, dtype: int64

In [232]:
X_train,X_test,y_train,y_test=train_test_split(X_en_1,y_en_1,random_state=15,stratify=y_en_1,test_size=0.2)

In [233]:
y_pred2=ANN(X_train,y_train,X_test,y_test,'binary_crossentropy',-1)

Epoch 1/100
3258/3258 [==============================] - 0s 107us/sample - loss: 0.6897 - acc: 0.5319
Epoch 2/100
3258/3258 [==============================] - 0s 43us/sample - loss: 0.6559 - acc: 0.6179
Epoch 3/100
3258/3258 [==============================] - 0s 44us/sample - loss: 0.6284 - acc: 0.6578
Epoch 4/100
3258/3258 [==============================] - 0s 44us/sample - loss: 0.6173 - acc: 0.6593
Epoch 5/100
3258/3258 [==============================] - 0s 45us/sample - loss: 0.6025 - acc: 0.6777
Epoch 6/100
3258/3258 [==============================] - 0s 44us/sample - loss: 0.5985 - acc: 0.6765
Epoch 7/100
3258/3258 [==============================] - 0s 44us/sample - loss: 0.5854 - acc: 0.6900
Epoch 8/100
3258/3258 [==============================] - 0s 44us/sample - loss: 0.5801 - acc: 0.6971
Epoch 9/100
3258/3258 [==============================] - 0s 45us/sample - loss: 0.5772 - acc: 0.7072
Epoch 10/100
3258/3258 [==============================] - 0s 44us/sample - loss: 0.5779 - 

3258/3258 [==============================] - 0s 45us/sample - loss: 0.3825 - acc: 0.8217
Epoch 82/100
3258/3258 [==============================] - 0s 43us/sample - loss: 0.3772 - acc: 0.8232
Epoch 83/100
3258/3258 [==============================] - 0s 42us/sample - loss: 0.3738 - acc: 0.8281
Epoch 84/100
3258/3258 [==============================] - 0s 43us/sample - loss: 0.3700 - acc: 0.8275
Epoch 85/100
3258/3258 [==============================] - 0s 43us/sample - loss: 0.3704 - acc: 0.8297
Epoch 86/100
3258/3258 [==============================] - 0s 42us/sample - loss: 0.3640 - acc: 0.8324
Epoch 87/100
3258/3258 [==============================] - 0s 42us/sample - loss: 0.3669 - acc: 0.8284
Epoch 88/100
3258/3258 [==============================] - 0s 42us/sample - loss: 0.3630 - acc: 0.8312
Epoch 89/100
3258/3258 [==============================] - 0s 43us/sample - loss: 0.3682 - acc: 0.8269
Epoch 90/100
3258/3258 [==============================] - 0s 41us/sample - loss: 0.3625 - acc: 

In [234]:
X_en_2,y_en_2=get_train_batch(df3_class0,df3_class1,4075,6112)
y_en_2.value_counts()

1    2037
0    2037
Name: Exited, dtype: int64

In [260]:
X_train,X_test,y_train,y_test=train_test_split(X_en_2,y_en_2,random_state=15,stratify=y_en_2,test_size=0.2)

In [236]:
y_pred3=ANN(X_train,y_train,X_test,y_test,'binary_crossentropy',-1)

Epoch 1/100
3259/3259 [==============================] - 0s 110us/sample - loss: 0.6677 - acc: 0.6008
Epoch 2/100
3259/3259 [==============================] - 0s 44us/sample - loss: 0.6156 - acc: 0.6698
Epoch 3/100
3259/3259 [==============================] - 0s 45us/sample - loss: 0.6110 - acc: 0.6735
Epoch 4/100
3259/3259 [==============================] - 0s 46us/sample - loss: 0.5800 - acc: 0.7042
Epoch 5/100
3259/3259 [==============================] - 0s 45us/sample - loss: 0.5681 - acc: 0.7100
Epoch 6/100
3259/3259 [==============================] - 0s 45us/sample - loss: 0.5424 - acc: 0.7327
Epoch 7/100
3259/3259 [==============================] - 0s 46us/sample - loss: 0.5374 - acc: 0.7370
Epoch 8/100
3259/3259 [==============================] - 0s 46us/sample - loss: 0.5236 - acc: 0.73830s - loss: 0.5212 - acc: 0.743
Epoch 9/100
3259/3259 [==============================] - 0s 46us/sample - loss: 0.5049 - acc: 0.7564
Epoch 10/100
3259/3259 [==============================] - 0s

3259/3259 [==============================] - 0s 44us/sample - loss: 0.3215 - acc: 0.8573
Epoch 82/100
3259/3259 [==============================] - 0s 45us/sample - loss: 0.3217 - acc: 0.8564
Epoch 83/100
3259/3259 [==============================] - 0s 44us/sample - loss: 0.3278 - acc: 0.8478
Epoch 84/100
3259/3259 [==============================] - 0s 45us/sample - loss: 0.3171 - acc: 0.8601
Epoch 85/100
3259/3259 [==============================] - 0s 45us/sample - loss: 0.3224 - acc: 0.8582
Epoch 86/100
3259/3259 [==============================] - 0s 44us/sample - loss: 0.3178 - acc: 0.8592
Epoch 87/100
3259/3259 [==============================] - 0s 43us/sample - loss: 0.3046 - acc: 0.8622
Epoch 88/100
3259/3259 [==============================] - 0s 42us/sample - loss: 0.3103 - acc: 0.8552
Epoch 89/100
3259/3259 [==============================] - 0s 44us/sample - loss: 0.3091 - acc: 0.8604
Epoch 90/100
3259/3259 [==============================] - 0s 43us/sample - loss: 0.2993 - acc: 

In [237]:
X_en_3,y_en_3=get_train_batch(df3_class0,df3_class1,6112,7963)
y_en_3.value_counts()

1    2037
0    1851
Name: Exited, dtype: int64

In [238]:
X_train,X_test,y_train,y_test=train_test_split(X_en_3,y_en_3,random_state=15,stratify=y_en_3,test_size=0.2)

In [239]:
y_pred4=ANN(X_train,y_train,X_test,y_test,'binary_crossentropy',-1)

Epoch 1/100
3110/3110 [==============================] - 0s 112us/sample - loss: 0.6637 - acc: 0.5997
Epoch 2/100
3110/3110 [==============================] - 0s 46us/sample - loss: 0.6241 - acc: 0.6566
Epoch 3/100
3110/3110 [==============================] - 0s 46us/sample - loss: 0.5942 - acc: 0.6839
Epoch 4/100
3110/3110 [==============================] - 0s 46us/sample - loss: 0.5624 - acc: 0.7048
Epoch 5/100
3110/3110 [==============================] - 0s 47us/sample - loss: 0.5362 - acc: 0.7264
Epoch 6/100
3110/3110 [==============================] - 0s 46us/sample - loss: 0.5279 - acc: 0.7341
Epoch 7/100
3110/3110 [==============================] - 0s 47us/sample - loss: 0.5103 - acc: 0.7424
Epoch 8/100
3110/3110 [==============================] - 0s 47us/sample - loss: 0.4992 - acc: 0.7534
Epoch 9/100
3110/3110 [==============================] - 0s 47us/sample - loss: 0.4916 - acc: 0.7614
Epoch 10/100
3110/3110 [==============================] - 0s 46us/sample - loss: 0.4864 - 

In [257]:
len(y_test)

778

In [261]:
y_pred_final=y_pred1.copy()

for i in range(len(y_pred1)):
    n_ones=y_pred1[i]+y_pred2[i]+y_pred3[i]
    if n_ones>1:
        y_pred_final[i]=1
    else:
        y_pred_final[i]=0

y_test.shape 

(815,)

In [262]:
print(classification_report(y_test,y_pred_final))

              precision    recall  f1-score   support

           0       0.75      0.88      0.81       408
           1       0.86      0.71      0.78       407

    accuracy                           0.80       815
   macro avg       0.81      0.80      0.79       815
weighted avg       0.81      0.80      0.79       815

